In [21]:
# Standard libraries
import pandas as pd
import numpy as np
from scipy import stats
# Visualization
import matplotlib.pyplot as plt
import datetime
import os 

# os.system("pip install wrds") # TODO: Probably put this in utils.py
import wrds

# os.system("pip install pandas-datareader")
import pandas_datareader.data as web

<b>TODO LIST:</b>
1. Parse audit analytics data for year and month of date of disclosure (possibly include date of breach) - Might have to select a couple of firms that we want to analyze?
    * Use year, month, and ticker to find stock price of company for that month (in WorldScope database)
    * Find stock price 1 to 12 months from that date (WorldScope). Calculate gain/loss in stock price from original. Potentially even take stock price the month before the breach?
2. Perform statistical tests + data visualization/analysis. Some examples:
    * Regression on multiple variables to predict stock price gain/loss over x months 
    * Scatter plot over time (ex: 2000-2021) of stock price gains/losses
    * Scatter plot over time after date of disclosure/breach of stock price gains/losses
    * Use Regression to see if type of breach matters for stock price
    * Regression to predict records lost?
    * Create decision tree to see type of breach

<b>Problems to maybe deal with:</b> <br>
What to do with 0 total records? - drop or use some averaging technique?
Do we combine audit analytics dataset with another one (PRC? Gordon said the PRC data is more erroneous than audit analytics, audit analytics more trusted) - make sure no repeats

<b>Resources for WRDS:</b> <br>
WorldScope (I think this is what we want): 
    https://wrds-web.wharton.upenn.edu/wrds//ds/tfn/ws/stock/index.cfm#variablesTab
    https://wrds-web.wharton.upenn.edu/wrds//ds/compd/secm/index.cfm
* Use: 5001 -- Market Price - Year End, stuff like 5015 -- End Market Price - January, 5601 -- ticker, 6038 -- IBES ticker

CRSP also works but I think WorldScope is better.
https://wrds-web.wharton.upenn.edu/wrds//ds/crsp/stock_a/msf.cfm
https://wrds-web.wharton.upenn.edu/wrds/tools/variable.cfm?library_id=20&file_id=123976


<b>There already seems to be a nice data reader for this! https://www.youtube.com/watch?v=57qAxRV577c USe this instead maybe?, stop using other stuff</b><br>

In [2]:
db = wrds.Connection()

Enter your WRDS username [gabri]:gnaval
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [3]:
# Example WRDS code
# db.list_tables('crsp')
db.get_table(library = 'crsp', table ='msf', obs = 10)

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,ret,bid,ask,shrout,cfacpr,cfacshr,altprc,spread,altprcdt,retx
0,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1985-12-31,NaN,NaN,NaN,...,NaN,None,None,NaN,NaN,NaN,-2.562500,NaN,1986-01-07,NaN
1,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-01-31,-2.50000,-4.437500,-4.375000,...,NaN,None,None,3680.0,1.0,1.0,-4.375000,0.25000,1986-01-31,NaN
2,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-02-28,-3.25000,-4.375000,-3.250000,...,-0.257143,None,None,3680.0,1.0,1.0,-3.250000,0.25000,1986-02-28,-0.257143
3,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-03-31,-3.25000,-4.437500,-4.437500,...,0.365385,None,None,3680.0,1.0,1.0,-4.437500,0.12500,1986-03-31,0.365385
4,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-04-30,-4.00000,-4.312500,-4.000000,...,-0.098592,None,None,3793.0,1.0,1.0,-4.000000,0.25000,1986-04-30,-0.098592
5,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-05-30,-3.06250,-4.218750,-3.109375,...,-0.222656,None,None,3793.0,1.0,1.0,-3.109375,0.09375,1986-05-30,-0.222656
6,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-06-30,-2.90625,-3.296875,-3.093750,...,-0.005025,None,None,3793.0,1.0,1.0,-3.093750,0.06250,1986-06-30,-0.005025
7,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-07-31,-2.59375,-3.437500,-2.843750,...,-0.080808,None,None,3793.0,1.0,1.0,-2.843750,0.06250,1986-07-31,-0.080808
8,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-08-29,-1.03125,-2.625000,-1.093750,...,-0.615385,None,None,3793.0,1.0,1.0,-1.093750,0.06250,1986-08-29,-0.615385
9,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-09-30,-0.96875,-1.281250,-1.031250,...,-0.057143,None,None,3793.0,1.0,1.0,-1.031250,0.06250,1986-09-30,-0.057143


In [4]:
# Example WRDS code
# data_5rows = db.get_table(library='wrdssec', table='wrds_nlpsa', columns = ['gvkey', 'cik', 'filename', 'finterms_litigious_count'], obs=5)
pd.set_option('display.max_rows', 10)

sql_query="""
SELECT *
FROM trws.wrds_ws_stock
WHERE ITEM5601 = 'GEO' AND year_=2020
"""
data_query = db.raw_sql(sql_query)
display(data_query)
# data_query[data_query['item6038'].str.contains('CFX') == True]


,code,year_,freq,seq,date_,item5001,item5002,item5003,item5004,item5005,...,item18193,item18202,item18205,item18208,item18209,item18210,item18211,item18259,item18260,item18261
0,19875.0,2020.0,A,1.0,2020-10-30,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,NaN,None,None,None,NaN,NaN
1,57330.0,2020.0,A,1.0,2020-10-30,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,NaN,None,None,None,NaN,NaN
2,123107.0,2020.0,A,1.0,2020-10-30,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,NaN,None,None,None,NaN,NaN
3,126954.0,2020.0,A,1.0,2020-10-30,0.06,0.127,0.02,0.45103,0.20213,...,None,None,None,None,-0.0252,None,None,None,0.0,0.0
4,127140.0,2020.0,A,1.0,2020-10-30,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,NaN,None,None,None,NaN,NaN


In [5]:
# PRC Dataset 
PRC_df = pd.read_csv("../data/prc.csv" )

# Data Cleaning
PRC_df.drop(PRC_df.columns[[13,14,15]], axis=1, inplace = True)
PRC_df = PRC_df[PRC_df['Total Records'].notna()]

def records_to_int(record_str):
    return int(str(record_str).replace(",", ""))

PRC_df['Total Records'] = PRC_df['Total Records'].apply(records_to_int)

# Drop rows with 0 total records (Maybe toggle this off we want to fill in data)
PRC_df = PRC_df[PRC_df['Total Records'] != 0]

In [18]:
# Audit Analytics Dataset
xls = pd.ExcelFile('../data/audit_analytics.xlsx')
aa_records_df = pd.read_excel(xls, 'PublicCyber')
aa_ddos_df = pd.read_excel(xls, 'DDoS')

# Get rid of rows with no tickers (these might be non-publicly traded)
aa_records_df = aa_records_df[aa_records_df['Ticker'].isna() != True]

In [15]:
# Ticker - Don't think we need this 
ticker_df = pd.read_csv("../data/ticker.csv" )
ticker_df

,Unnamed: 0,cik_str,ticker,title
0,0,320193,AAPL,Apple Inc.
1,1,789019,MSFT,MICROSOFT CORP
2,2,1018724,AMZN,AMAZON COM INC
3,3,1652044,GOOG,Alphabet Inc.
4,4,1293451,TCEHY,Tencent Holdings Ltd
...,...,...,...,...
11351,11351,1830081,CFVIU,CF Acquisition Corp. VI
11352,11352,1830063,NAACU,North Atlantic Acquisition Corp
11353,11353,1829889,IIIIW,INSU ACQUISITION CORP III
11354,11354,1829999,FOXWU,FoxWayne Enterprises Acquisition Corp.


In [34]:
test_df = aa_records_df[aa_records_df['Year'] < 2021]

,Company name,Related party,Target relationship to Parent,CIK,Ticker,Market,IRS Number,Parent CIK,Parent Name,Bus Street 1,Bus Street 2,City,County,State Code,State Name,Zip,Region,Bus Phone,Inc. State Code,Inc. State Name,SIC Code,SIC Description,NAICS Code,NAICS Description,Year,Month,Breach No.,Registration Number - Auditor Date of Breach,Auditor Date of Breach,Auditor Key Date of Breach,Auditor Affiliate Date of Breach,Registration Number - Auditor Affiliate Date of Breach,Auditor City,Auditor State Code,Auditor State Name,Auditor State Region,Registration Number - Current Auditor,Current Auditor,Current Auditor Key,Current Affiliate Auditor,Registration Number - Current Affiliate Auditor,Auditor City.1,Auditor State Code.1,Auditor State Name.1,Auditor State Region.1,Date of Breach,Date Became Aware of Breach,Date of Disclosure,Type of Breach,Number of Records,Type of Info,Information,Attack,Link,Source,Release,6/8-K SEC notification,Date,First Estimate Filing,Section,Filing Date,Filing Disclosure,Cost/Estimate,Reimbursements,Insurance,Litigation Docket,Lead Defendant Representation,Lead Plaintiff Representation,Litigation Settlement
33,"T-Mobile US, Inc.","T-Mobile US, Inc.",Parent,1283699,TMUS,Nasdaq Global Market,20-0836269,NaN,NaN,8144 WALNUT HILL LANE,SUITE 800,DALLAS,Dallas,TX,TEXAS,75231,US Southwest,(800) 318-9270,DE,DELAWARE,4812.0,Radiotelephone Communications,517212.0,Cellular and Other Wireless Telecommunications,2020,12,851,238.0,PricewaterhouseCoopers LLP,1.0,NaN,NaN,Seattle,WA,WASHINGTON,US West,238.0,PricewaterhouseCoopers LLP,1.0,,NaN,Seattle,WA,WASHINGTON,US West,NaN,NaN,2020-12-30,HACK,200000,Personal,Phone Number | Other,Unauthorized Access,https://www.bleepingcomputer.com/news/security...,Bleeping Computer,T-Mobile has announced a data breach exposing ...,https://www.sec.gov/Archives/edgar/data/128369...,2021-02-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,WHIRLPOOL CORP /DE/,WHIRLPOOL CORP /DE/,Parent,106640,WHR,NYSE,38-1490038,NaN,NaN,WHIRLPOOL CNTR 2000 M 63,C/O CORPORATE SECRETARY,BENTON HARBOR,Berrien,MI,MICHIGAN,49022-2692,US Midwest,(269) 923-5000,DE,DELAWARE,3630.0,Household Appliances,335228.0,Other Major Household Appliance Manufacturing,2020,12,850,42.0,Ernst & Young LLP,2.0,NaN,NaN,Chicago,IL,ILLINOIS,US Midwest,42.0,Ernst & Young LLP,2.0,,NaN,Chicago,IL,ILLINOIS,US Midwest,NaN,NaN,2020-12-28,HACK,NaN,Personal,Other,Ransomware,https://www.bleepingcomputer.com/news/security...,Bleeping Computer,Home appliances giant Whirlpool suffered a ran...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,Colfax CORP,"DJO Global, Inc.",Subsidiary,1420800,CFX,NYSE,54-1887631,NaN,NaN,420 NATIONAL BUSINESS PARKWAY,5TH FLOOR,ANNAPOLIS JUNCTION,Howard,MD,MARYLAND,20701,US Mid Atlantic,(301) 323-9000,DE,DELAWARE,3561.0,Pumps and Pumping Equipment,333911.0,Pump and Pumping Equipment Manufacturing,2020,12,849,42.0,Ernst & Young LLP,2.0,NaN,NaN,Baltimore,MD,MARYLAND,US Mid Atlantic,42.0,Ernst & Young LLP,2.0,,NaN,Baltimore,MD,MARYLAND,US Mid Atlantic,11/8/2020 - 11/9/2020,2020-11-16 00:00:00,2020-12-23,HACK,NaN,Financial,Name|SSN|Other|Bank Account,Unauthorized Access,https://oag.ca.gov/system/files/DJO%20-%20Noti...,Office of Attorney General: California,We write to share important information with y...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,"VMWARE, INC.",SolarWinds Corp,Third-Party,1124610,VMW,NYSE,94-3292913,NaN,NaN,3401 HILLVIEW AVENUE,NaN,PALO ALTO,Santa Clara,CA,CALIFORNIA,94304,US West,(650) 427-5000,DE,DELAWARE,7372.0,Prepackaged Software,541511.0,Custom Computer Programming Services,2020,12,848,238.0,PricewaterhouseCoopers LLP,1.0,NaN,NaN,San Jose,CA,CALIFORNIA,US West,238.0,PricewaterhouseCoopers LLP,1.0,,NaN,San Jose,CA,CALIFORNIA,US West,NaN,NaN,2020-12-21,HACK,NaN,ND,ND,Malware,https://www.wsj.com/articles/solarwinds-hack-v...,Wall Street Journal,The suspected Russian hackers behind breaches ...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,NVIDIA CORP,SolarWinds C

In [77]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs((x - pivot).days))

def stock_after_disclosure(row, months_after):
    start = pd.to_datetime(row['Date of Disclosure']) 
    end = start + pd.DateOffset(months = months_after)
    # Don't know if i should include this, check stock day before breach to control for large stock dip when breach is disclosed
    start -= datetime.timedelta(days=1) 
    df = web.DataReader(row['Ticker'], 'yahoo', start, end)
    display(df)
    lst = []
    for month in range(0, months_after + 1):
        date = nearest(df.index, (start + pd.DateOffset(months = month)))
        print(df.loc[date]["Close"])
        lst.append(df.loc[date]["Close"])
    return lst
    
# start = pd.to_datetime("4/28/2020") - datetime.timedelta(days=1)
# end = start + pd.DateOffset(months = 5)
# chegg = web.DataReader("CHGG", 'yahoo', start, end)
# chegg

In [78]:
# Minitest
a = test_df.head(3)
lst = []
for index, row in a.iterrows():
    x = stock_after_disclosure(row, 1)
    lst.append(x)
    
print(lst)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-12-29,133.970001,131.649994,133.520004,133.139999,1672500,133.139999
2020-12-30,133.830002,132.509995,133.520004,133.279999,1673100,133.279999
2020-12-31,135.000000,133.100006,133.149994,134.850006,2080600,134.850006
2021-01-04,134.979996,130.460007,134.300003,132.429993,3433100,132.429993
2021-01-05,132.240005,130.460007,131.949997,131.850006,2544200,131.850006
...,...,...,...,...,...,...
2021-01-25,132.399994,129.669998,130.750000,131.100006,2892600,131.100006
2021-01-26,132.710007,130.000000,130.020004,131.410004,3219000,131.410004
2021-01-27,130.000000,122.449997,129.690002,123.269997,7023100,123.269997


133.13999938964844
126.08000183105469
[133.13999938964844, 126.08000183105469]


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-12-28,194.000000,185.119995,193.869995,185.190002,571800,183.993805
2020-12-29,186.330002,181.300003,185.199997,183.199997,399200,182.016647
2020-12-30,185.830002,183.690002,184.149994,184.389999,373100,183.198975
2020-12-31,183.929993,180.259995,183.929993,180.490005,535000,179.324173
2021-01-04,182.690002,175.369995,182.479996,176.899994,886800,175.757339
...,...,...,...,...,...,...
2021-01-22,198.320007,191.539993,193.500000,198.210007,599500,196.929703
2021-01-25,210.779999,199.570007,199.570007,203.020004,1150100,201.708633
2021-01-26,208.199997,201.190002,203.029999,206.419998,756700,205.086670


185.19000244140625
212.39999389648438
[185.19000244140625, 212.39999389648438]


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-12-22,36.750000,35.840000,36.619999,36.189999,379900,36.189999
2020-12-23,37.169998,36.380001,36.560001,36.650002,650500,36.650002
2020-12-24,36.869999,36.320000,36.779999,36.790001,117700,36.790001
2020-12-28,37.490002,36.900002,37.250000,36.900002,444700,36.900002
2020-12-29,37.310001,36.320000,37.240002,36.680000,464800,36.680000
...,...,...,...,...,...,...
2021-01-15,39.389999,37.910000,39.180000,38.730000,985300,38.730000
2021-01-19,39.950001,39.110001,39.270000,39.590000,866500,39.590000
2021-01-20,40.759998,39.599998,39.810001,40.110001,796400,40.110001


36.189998626708984
40.599998474121094
[36.189998626708984, 40.599998474121094]
[[133.13999938964844, 126.08000183105469], [185.19000244140625, 212.39999389648438], [36.189998626708984, 40.599998474121094]]


In [107]:
a = test_df.head(3).reset_index()
b = pd.DataFrame(lst, columns = ['x', 'y'])
# display(a)
# display(b)
concatenated = pd.concat([a, b], axis=1, join='inner')
display(concatenated)

,index,Company name,Related party,Target relationship to Parent,CIK,Ticker,Market,IRS Number,Parent CIK,Parent Name,Bus Street 1,Bus Street 2,City,County,State Code,State Name,Zip,Region,Bus Phone,Inc. State Code,Inc. State Name,SIC Code,SIC Description,NAICS Code,NAICS Description,Year,Month,Breach No.,Registration Number - Auditor Date of Breach,Auditor Date of Breach,Auditor Key Date of Breach,Auditor Affiliate Date of Breach,Registration Number - Auditor Affiliate Date of Breach,Auditor City,Auditor State Code,Auditor State Name,Auditor State Region,Registration Number - Current Auditor,Current Auditor,Current Auditor Key,Current Affiliate Auditor,Registration Number - Current Affiliate Auditor,Auditor City.1,Auditor State Code.1,Auditor State Name.1,Auditor State Region.1,Date of Breach,Date Became Aware of Breach,Date of Disclosure,Type of Breach,Number of Records,Type of Info,Information,Attack,Link,Source,Release,6/8-K SEC notification,Date,First Estimate Filing,Section,Filing Date,Filing Disclosure,Cost/Estimate,Reimbursements,Insurance,Litigation Docket,Lead Defendant Representation,Lead Plaintiff Representation,Litigation Settlement,x,y
0,33,"T-Mobile US, Inc.","T-Mobile US, Inc.",Parent,1283699,TMUS,Nasdaq Global Market,20-0836269,NaN,NaN,8144 WALNUT HILL LANE,SUITE 800,DALLAS,Dallas,TX,TEXAS,75231,US Southwest,(800) 318-9270,DE,DELAWARE,4812.0,Radiotelephone Communications,517212.0,Cellular and Other Wireless Telecommunications,2020,12,851,238.0,PricewaterhouseCoopers LLP,1.0,NaN,NaN,Seattle,WA,WASHINGTON,US West,238.0,PricewaterhouseCoopers LLP,1.0,,NaN,Seattle,WA,WASHINGTON,US West,NaN,NaN,2020-12-30,HACK,200000,Personal,Phone Number | Other,Unauthorized Access,https://www.bleepingcomputer.com/news/security...,Bleeping Computer,T-Mobile has announced a data breach exposing ...,https://www.sec.gov/Archives/edgar/data/128369...,2021-02-23 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,133.139999,126.080002
1,34,WHIRLPOOL CORP /DE/,WHIRLPOOL CORP /DE/,Parent,106640,WHR,NYSE,38-1490038,NaN,NaN,WHIRLPOOL CNTR 2000 M 63,C/O CORPORATE SECRETARY,BENTON HARBOR,Berrien,MI,MICHIGAN,49022-2692,US Midwest,(269) 923-5000,DE,DELAWARE,3630.0,Household Appliances,335228.0,Other Major Household Appliance Manufacturing,2020,12,850,42.0,Ernst & Young LLP,2.0,NaN,NaN,Chicago,IL,ILLINOIS,US Midwest,42.0,Ernst & Young LLP,2.0,,NaN,Chicago,IL,ILLINOIS,US Midwest,NaN,NaN,2020-12-28,HACK,NaN,Personal,Other,Ransomware,https://www.bleepingcomputer.com/news/security...,Bleeping Computer,Home appliances giant Whirlpool suffered a ran...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,185.190002,212.399994
2,35,Colfax CORP,"DJO Global, Inc.",Subsidiary,1420800,CFX,NYSE,54-1887631,NaN,NaN,420 NATIONAL BUSINESS PARKWAY,5TH FLOOR,ANNAPOLIS JUNCTION,Howard,MD,MARYLAND,20701,US Mid Atlantic,(301) 323-9000,DE,DELAWARE,3561.0,Pumps and Pumping Equipment,333911.0,Pump and Pumping Equipment Manufacturing,2020,12,849,42.0,Ernst & Young LLP,2.0,NaN,NaN,Baltimore,MD,MARYLAND,US Mid Atlantic,42.0,Ernst & Young LLP,2.0,,NaN,Baltimore,MD,MARYLAND,US Mid Atlantic,11/8/2020 - 11/9/2020,2020-11-16 00:00:00,2020-12-23,HACK,NaN,Financial,Name|SSN|Other|Bank Account,Unauthorized Access,https://oag.ca.gov/system/files/DJO%20-%20Noti...,Office of Attorney General: California,We write to share important information with y...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.189999,40.599998


In [64]:
pd.set_option('display.max_rows', 10)

# display(chegg)
x = start + pd.DateOffset(months = 2)
nearest(chegg.index, x)

# chegg.loc[x]

Timestamp('2020-06-26 00:00:00')

In [14]:
# This is old code that attempted to use WORLDSCOPE (WRDS database) to figure out stock price. However, it kept on yield NAN values


def month_to_stock_col(month):
    return "ITEM" + str(5010 + 5 * month)


def find_initial_stock(row):
    sql_query = """
    SELECT {} AS init_price, ITEM5601 AS ticker, ITEM6038 AS IBES_ticker 
    FROM trws.wrds_ws_stock
    WHERE ITEM5601 ='{}' and year_={}
    """.format(month_to_stock_col(row['Month']), row['Ticker'], row['Year'])
    df = db.raw_sql(sql_query)
#     display(df)
    num_rows = len(df.index)
    if num_rows == 0:
        return np.nan
    elif num_rows == 1:
        return df['init_price'].iloc[0]
    else:
        print(df.columns)
        match_ibes_df = df[df['ibes_ticker'].str.contains(row['Ticker']) == True]
        print("Number of matching IBES tickers: %s" % len(match_ibes_df))
        num_rows = len(match_ibes_df)
        if num_rows == 0:
            print("No matching IBES tickers found")
            display(df)
        elif num_rows == 1:
            return match_ibes_df['init_price'].iloc[0]
        else:
            print("Multiple matching IBES tickers found")
            display(match_ibes_df)
    return np.nan

# sql_query="""
# SELECT *
# FROM trws.wrds_ws_stock
# WHERE year_=2021
# """

test_df['init_price'] = test_df.apply(lambda row: find_initial_stock(row), axis=1)
test_df['init_price']

Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1
Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 2
Multiple matching IBES tickers found


,init_price,ticker,ibes_ticker
3,None,CLH,CLHB
4,None,CLH,@CLHH


Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 0
No matching IBES tickers found


,init_price,ticker,ibes_ticker
0,None,SPOT,020K
1,None,SPOT,None


Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 0
No matching IBES tickers found


...

,init_price,ticker,ibes_ticker
0,18.2,TNL,None
1,18.2,TNL,@TNX


Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 0
No matching IBES tickers found


,init_price,ticker,ibes_ticker
0,14.8,TNL,None
1,14.8,TNL,@TNX


Index(['init_price', 'ticker', 'ibes_ticker'], dtype='object')
Number of matching IBES tickers: 1


TypeError: unsupported operand type(s) for *: 'int' and 'datetime.datetime'